# Examine trans effect of TP53 Hotspot mutation in Ovarian Cancer

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import cptac
import cptac.utils as ut

ov = cptac.Ovarian()

/anaconda3/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Use parse hotspot utility to analyze TP53 hotspot mutation in Ovarian cancer

#### Get somatic mutation data from cptac

In [2]:
somatic_mutations = ov.get_somatic_mutation()

In [3]:
somatic_mutations

Name,Gene,Mutation,Location
Patient_ID,,,
S002,ALPK1,Missense_Mutation,p.C39F
S002,AP4B1,Missense_Mutation,p.C420Y
S002,ARHGAP6,Missense_Mutation,p.E274V
S002,ARMC6,Missense_Mutation,p.L319V
S002,BAI2,Missense_Mutation,p.A229P
S002,BMS1,Missense_Mutation,p.D870G
S002,C10orf126,Missense_Mutation,p.F91L
S002,C15orf60,Missense_Mutation,p.D112E
S002,C2CD4A,In_Frame_Ins,p.D84delinsAN


#### Use parse hotspot utility

In [4]:
vis, binary, detailed, dictionary = ut.parse_hotspot('../../HotspotClusterFiles/ovarian_somatic_38.maf.3D_Proximity.pairwise.recurrence.l0.r10.clusters', somatic_mutations)

In [5]:
vis

,hotspot_id,patients_within
0,ADH1A,2.0
1,CDCA2,3.0
2,CPS1,2.0
3,EIF4A1,3.0
4,FGF18,3.0
5,GCA,2.0
6,GOT1,2.0
7,HNRNPM,3.0
8,KDM2A,2.0
9,KHK,2.0
